In [ ]:
import os
import numpy as np
import pandas as pd
import xlwings as xw
import openai
import ast

# OpenAI API 키 설정
# 실행 전, 아래에 실제 API 키를 입력하세요.
openai.api_key = "YOUR_API_KEY_HERE"

In [ ]:
# 리뷰 분류 함수 정의
# OpenAI API를 사용하여 리뷰를 속성별로 분류하고, 그 결과를 반환.
def review_classifier(review):
    
    # GPT API 호출을 위한 프롬프트 설정
    ask_messages=[
        {'role': 'system',
         'content': 'You are a review analyst who is Korean native speaker'
         },
        {'role': 'user',
         'content': "Please classify the sentences in the tablet review by attribute and evaluate whether the sentence is satisfied with the attribute. Please show me the Korean sentence that you didn't translate."
         },
        {'role': 'assistant',
         'content': "재질 is texture and touch-related content\n디자인 is design. It is about how it looks like. And it is not about color. \n색상 is color. It is about color! \n그립감 is aobut how does it feel when you hold it in your hand or about grip. not about touch\n서비스 is about delivery and A/S.\n휴대성 is about is it good to carry on\nQuality, product condition, and finishing processing  will go into 품질.\n구성 is about what's in this product and what's satisfactory."
         },
        {'role': 'user',
         'content': 'In the case of satisfaction, please mark satisfaction as 1, dissatisfaction as -1'
         },
        {'role': 'assistant',
         'content': 'Only find the properties in the review that I initially told u, and output it as a python list of [property, review content, positive/negative]. Please answer only with Python List. I do not need another word.'
         },
        {'role': 'user', 'content': f"the review is {review}"}
         ]
    
    # OpenAI ChatCompletion API 호출
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=ask_messages,
        temperature = 1.0,
        max_tokens = 2048,
        frequency_penalty = 0.0,
        presence_penalty = 1.0,
        top_p=1.0
    )
    
    # API 응답 결과 파싱
    gpt_answer = completion.choices[0].message.content

    # 문자열 형식의 응답을 Python 리스트로 변환
    try:
        result_answer = ast.literal_eval(gpt_answer)
    except:
        result_answer = []
    
    return result_answer

In [ ]:
# 원본 리뷰 데이터프레임 로드 (예시)
# df_reviews = pd.read_pickle('./reviews.pickle')
# df_reviews.head()

In [ ]:
# 전체 리뷰 개수를 100으로 나누어 처리할 청크 수 계산
len_hund = int(len(df_reviews) / 100 + 1)

# 결과를 저장할 리스트 및 데이터프레임 초기화
list_df_result = []
df_result_all = pd.DataFrame(columns=['review_number', 'review', 'classify_result'])

# 청크 단위로 반복 처리
for hundred_i in range(len_hund):
    # 결과 청크 데이터프레임의 이름을 리스트에 추가
    list_df_result.append(f"df_result_{hundred_i}")
    
    # 각 청크별 결과 저장을 위한 데이터프레임을 동적으로 생성
    globals()[f"df_result_{hundred_i}"] = pd.DataFrame(columns=['review_number', 'review', 'classify_result'])
    
    # 100개씩 리뷰를 처리
    for i in range(100):
        idx = hundred_i * 100 + i

        # 데이터프레임의 인덱스 범위를 벗어나는 경우, 반복 중단
        if idx >= len(df_reviews):
            break
        
        review_to_classify = df_reviews['리뷰'][idx]
        
        # 리뷰 분류 함수 호출 (오류 발생 시 NaN 처리)
        try:
            classify_result = review_classifier(review_to_classify)
        except:
            classify_result = np.nan
            
        # 분류 결과를 데이터프레임에 추가
        df_to_add = pd.DataFrame([{'review_number': idx, 'review': review_to_classify, 'classify_result': classify_result}])
        globals()[f"df_result_{hundred_i}"] = pd.concat([globals()[f"df_result_{hundred_i}"], df_to_add], ignore_index=True)
        df_result_all = pd.concat([df_result_all, df_to_add], ignore_index=True)

    # 100개 처리 단위로 중간 결과를 pickle 파일로 저장
    globals()[f"df_result_{hundred_i}"] .to_pickle(f'./classify_result/df_result_{hundred_i}.pickle')
    # 전체 결과도 pickle 파일로 저장
    df_result_all.to_pickle('./classify_result/df_result_all.pickle')